In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3
from sqlite3 import Error

In [3]:
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [4]:
connection = create_connection("FC724_PA2_RE.db")

Connection to SQLite DB successful


In [5]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

In [6]:
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")

In [10]:
a = """
SELECT * FROM Renewable_Energy_Adoption
WHERE Country = 'USA'
"""
execute_read_query(connection, a)

[('USA',
  '2000',
  '2383',
  '7.027075844154529',
  '279.0',
  '2.0633441172699056',
  '1.3551289306299015',
  '0.31624549434964416',
  '0.6147573892193059',
  '1.1954098372929238',
  '',
  '2.2537411132259058'),
 ('USA',
  '2001',
  '2734',
  '6.816378050843563',
  '384.0',
  '1.037638065294199',
  '0.7089880083852926',
  '1.2852448690152265',
  '0.0038532787624391157',
  '0.9065820381705672',
  '1.3373389554719213',
  '2.574370901038117'),
 ('USA',
  '2002',
  '1607',
  '9.538257946022448',
  '109.0',
  '2.220548554178556',
  '0.13787452709245485',
  '0.7815964265756689',
  '1.4918203055106842',
  '0.7459470309456466',
  '1.4152212937361996',
  '4.965798362161793'),
 ('USA',
  '2003',
  '3587',
  '4.334616149281134',
  '282.0',
  '0.7246723457307672',
  '0.6019055146184645',
  '0.7584585898379662',
  '0.6136200169175781',
  '0.365135326984724',
  '0.7642921180184239',
  '1.2312045829039766'),
 ('USA',
  '2004',
  '2363',
  '',
  '161.0',
  '0.8568329603337053',
  '',
  '0.443609906